In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas_ta as ta
from arch import arch_model
import numpy as np
from ib_insync import *
from collections import deque
from enum import Enum
import datetime
import logging
import math
util.startLoop()

In [ ]:
"""ToDos
1) functionality to add multiple contracts to RealTimeData
1) indicator functions - use MA and EMA from previous version.
    - should be able to submit either backtest DF or a realtime DF
    
2) Rule functions - when is a buy or sell signal generated
3) copy over account class








"""

In [5]:
"""from chat-GPT"""
import ib_insync
import pandas as pd

# Connect to the Interactive Brokers TWS/IB Gateway
ib = ib_insync.IB()
ib.connect('127.0.0.1', 7497, 3)

# Define the assets to be traded
# assets = ['AAPL', 'GOOG', 'MSFT', 'FB']
assets = ['NQ']

# Define the moving average window
window = 5

# Get the historical data for the assets
df = pd.DataFrame()
for asset in assets:
#     contract = ib_insync.Stock(asset, 'SMART', 'USD')
    contract = ib_insync.Future(asset,  '20230317', 'CME')
    bars = ib.reqHistoricalData(contract, endDateTime='', durationStr='180 D', barSizeSetting='1 day', whatToShow='TRADES', useRTH=True)
    df[asset] = [bar.close for bar in bars]

# Calculate the moving average for each asset
for asset in assets:
    df[f"{asset}_MA_{window}"] = df[asset].rolling(window=window).mean()

# Buy and sell logic
for asset in assets:
    # Buy when the price crosses above the moving average
    if df[asset].iloc[-1] > df[f"{asset}_MA_{window}"].iloc[-1] and df[asset].iloc[-2] < df[f"{asset}_MA_{window}"].iloc[-2]:
        print(f"Buy {asset}")
    
    # Sell when the price crosses below the moving average
    elif df[asset].iloc[-1] < df[f"{asset}_MA_{window}"].iloc[-1] and df[asset].iloc[-2] > df[f"{asset}_MA_{window}"].iloc[-2]:
        print(f"Sell {asset}")
    else:
        print('none')

# Disconnect from the Interactive Brokers TWS/IB Gateway
ib.disconnect()


none


In [6]:
df

,NQ,NQ_MA_5
0,12063.50,NaN
1,12400.75,NaN
2,12803.00,NaN
3,12770.50,NaN
4,12679.50,12543.45
...,...,...
169,12221.00,12012.15
170,11988.75,12022.80
171,12119.25,12052.60
172,12505.25,12178.95


In [54]:
def BacktestData(ib, contractList, duration, barSize):
    df_list=[]
    for contract in contractList:
        bars = ib.reqHistoricalData(
            contract, endDateTime='', durationStr=duration,
            barSizeSetting=barSize, whatToShow='MIDPOINT', useRTH=True)

        df = util.df(bars)
        df = df[['date','close']]
        df = df.rename({'close':contract.symbol}, axis=1)
        df_list.append(df)

    if len(df_list)==1:
        return df_list[0]
    
    df_final = df_list[0]
    for i in range(1,len(df_list)):
        df_final = pd.merge(df_final, df_list[i], on='date', how='inner')
    return df_final

"""test"""
ib2 = IB()
ib2.connect('127.0.0.1', 7497, 1)
futures = [Future('NQ', '20230317', 'CME'), Future('ES', '20230317', 'CME')]
duration = '180 D'
barSize = '1 day'

df = BacktestData(ib2, futures, duration, barSize)

In [2]:
def MovingAverage(df, low=5, high=10):
    df[str(low)+'D MA'] = df['Close'].rolling(low).mean()
    df[str(high)+'D MA'] = df['Close'].rolling(high).mean()
    return df


class RealTimeData():
    def __init__(self):
        pass
    
    def getDataFrame(self):
        self.df = pd.DataFrame(columns=['Date', 'Close'])    
    
    def addData(self, bar):
        self.df = self.df.append({'Date':bar.time.date(), 'Close':bar.close}, ignore_index=True)
        display(self.df)
        
#         self.processData()

    def onBarUpdate(self, bars, newBar):
        if newBar:
            print('on bar update', bars[-1])
            lst.append(bars[-1])   

    def processData(self):
        pass
        
    
    def addDataDF(self, bars, newBar):
        if newBar:
            self.addData(bars[-1])






In [32]:
contract1 = Crypto('BTC', exchange='PAXOS', currency='USD')
contract2 = Crypto('ETH', exchange='PAXOS', currency='USD')
contracts = [contract1, contract2]
ib = IB()
ib.connect('127.0.0.1', 7497, 7)

data = RealTimeData()
data.getDataFrame()
bar_list=[]
for contract in contracts:
    bars = ib.reqRealTimeBars(contract=contract,barSize=5,whatToShow='MIDPOINT',useRTH=False)
#     bars.updateEvent += data.onBarUpdate
    bars.updateEvent += data.addDataDF
    ib.sleep(5)
#     bar_list.append(bars)
#     ib.cancelRealTimeBars(bars)
#     df = MovingAverage(data.df, 5, 10)
ib.disconnect()

,Date,Close
0,2023-02-16,24694.875


,Date,Close
0,2023-02-16,24694.875
1,2023-02-16,24694.875


,Date,Close
0,2023-02-16,24694.875
1,2023-02-16,24694.875
2,2023-02-16,1692.575


,Date,Close
0,2023-02-16,24694.875
1,2023-02-16,24694.875
2,2023-02-16,1692.575
3,2023-02-16,1692.575


,Date,Close
0,2023-02-16,24694.875
1,2023-02-16,24694.875
2,2023-02-16,1692.575
3,2023-02-16,1692.575
4,2023-02-16,24694.875


In [6]:
bar_list

[[RealTimeBar(time=datetime.datetime(2023, 2, 16, 3, 55, 40, tzinfo=datetime.timezone.utc), endTime=-1, open_=24722.375, high=24722.375, low=24722.375, close=24722.375, volume=-1.0, wap=-1.0, count=-1),
  RealTimeBar(time=datetime.datetime(2023, 2, 16, 3, 55, 45, tzinfo=datetime.timezone.utc), endTime=-1, open_=24722.375, high=24722.375, low=24722.375, close=24722.375, volume=-1.0, wap=-1.0, count=-1),
  RealTimeBar(time=datetime.datetime(2023, 2, 16, 3, 55, 50, tzinfo=datetime.timezone.utc), endTime=-1, open_=24722.375, high=24722.375, low=24722.375, close=24722.375, volume=-1.0, wap=-1.0, count=-1)],
 [RealTimeBar(time=datetime.datetime(2023, 2, 16, 3, 55, 45, tzinfo=datetime.timezone.utc), endTime=-1, open_=1691.975, high=1691.975, low=1691.975, close=1691.975, volume=-1.0, wap=-1.0, count=-1),
  RealTimeBar(time=datetime.datetime(2023, 2, 16, 3, 55, 50, tzinfo=datetime.timezone.utc), endTime=-1, open_=1691.975, high=1691.975, low=1691.975, close=1691.975, volume=-1.0, wap=-1.0, cou

In [4]:
df

,Date,Close,5D MA,10D MA
0,2023-02-16,24707.125,NaN,NaN
1,2023-02-16,24707.125,NaN,NaN
2,2023-02-16,24707.125,NaN,NaN
3,2023-02-16,24707.125,NaN,NaN
4,2023-02-16,24707.125,24707.125,NaN
5,2023-02-16,24707.125,24707.125,NaN
6,2023-02-16,24707.125,24707.125,NaN
7,2023-02-16,24709.375,24707.575,NaN
8,2023-02-16,24709.375,24708.025,NaN
9,2023-02-16,24709.375,24708.475,24707.800


In [18]:
# ib.cancelRealTimeBars(bars)

In [19]:
# data.df

In [20]:
# ib.cancelRealTimeBars(bars)        

Peer closed connection.


In [59]:
"""data_df is either historical data from backtestdat"""
def RunAlgo(ib, data_df, contractList, duration, barSize, indicatorList):
    pass
    
        


In [ ]:
import asyncio

import ib_insync as ibi


class App:

    async def run(self):
        self.ib = ibi.IB()
        with await self.ib.connectAsync():
            contracts = [
#                 ibi.Stock(symbol, 'SMART', 'USD')
                 ibi.Crypto(symbol, exchange='PAXOS', currency='USD')
                for symbol in ['BTC', 'ETH']
            for contract in contracts:
                self.ib.reqMktData(contract)

            async for tickers in self.ib.pendingTickersEvent:
                for ticker in tickers:
                    print(ticker)

    def stop(self):
        self.ib.disconnect()


app = App()
try:
    asyncio.run(app.run())
except (KeyboardInterrupt, SystemExit):
    app.stop()

In [ ]:
import ib_insync as ibi
import pygame


def onTicker(ticker):
    screen.fill(bg_color)
    text = f'bid: {ticker.bid}   ask: {ticker.ask}'
    quote = font.render(text, True, fg_color)
    screen.blit(quote, (40, 40))
    pygame.display.flip()


pygame.init()
screen = pygame.display.set_mode((800, 600))
font = pygame.font.SysFont('arial', 48)
bg_color = (255, 255, 255)
fg_color = (0, 0, 0)

ib = ibi.IB()
ib.connect()
contract = ibi.Forex('EURUSD')
ticker = ib.reqMktData(contract)
ticker.updateEvent += onTicker

running = True
while running:
    # This updates IB-insync:
    ib.sleep(0.03)

    # This updates PyGame:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()

In [15]:
"""threading example from chatgpt"""

from ib_insync import *
import threading

# create a list of contracts
# contracts = [Contract(symbol='AAPL', exchange='SMART', currency='USD'),
#              Contract(symbol='GOOG', exchange='SMART', currency='USD'),
#              Contract(symbol='MSFT', exchange='SMART', currency='USD')]

contract1 = Crypto('BTC', exchange='PAXOS', currency='USD')
contract2 = Crypto('ETH', exchange='PAXOS', currency='USD')
contracts = [contract1, contract2]


# create a list of threads
threads = []

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

# define a function to get data for a contract
def get_data(contract):
    
    
    ib.reqTickByTickData(contract, 'AllLast', 0, False)
    ib.sleep(5)
    ib.disconnect()

# create a thread for each contract
for contract in contracts:
    t = threading.Thread(target=get_data, args=(contract,))
    threads.append(t)
    t.start()

# wait for all the threads to complete
for t in threads:
    t.join()

# process the data for all contracts
data = []
for contract in contracts:
    data += ib.reqTickByTickData(contract, 'AllLast')
# process_data(data)


In [30]:
from ib_insync import *

# contracts = [
#     Stock('AAPL', 'SMART', 'USD'),
#     Future('ES', '20210319', 'GLOBEX'),
#     Option('SPY', '20210319', 400, 'C', 'SMART', multiplier=100)
# ]

contract1 = Crypto('BTC', exchange='PAXOS', currency='USD')
contract2 = Crypto('ETH', exchange='PAXOS', currency='USD')
contracts = [contract1, contract2]


ib = IB()
ib.connect('localhost', 7497, clientId=10)

tickers = [ib.reqMktData(contract, '', False, False) for contract in contracts]
data = util.df(tickers)
# data = util.df([t.last for t in tickers], index=[t.contract for t in tickers], columns=['lastPrice'])
def on_tick_price(ticker, tickType, price, attrib):
    data.loc[ticker.contract, tickType] = price

for ticker in tickers:
    ticker.tickPriceEvent += on_tick_price
print(data)


AttributeError: 'Ticker' object has no attribute 'tickPriceEvent'

In [31]:
tickers

[Ticker(contract=Crypto(symbol='BTC', exchange='PAXOS', currency='USD'), time=datetime.datetime(2023, 2, 16, 4, 16, 10, 505369, tzinfo=datetime.timezone.utc), bid=24664.25, bidSize=1.0, ask=24664.5, askSize=2.0, last=24664.5, lastSize=1.0, prevLast=24664.25, volume=219.66801174, high=24904.0, low=24012.75, close=24143.25, halted=0.0, ticks=[TickData(time=datetime.datetime(2023, 2, 16, 4, 16, 10, 505369, tzinfo=datetime.timezone.utc), tickType=8, price=-1.0, size=219.66801174)]),
 Ticker(contract=Crypto(symbol='ETH', exchange='PAXOS', currency='USD'), time=datetime.datetime(2023, 2, 16, 4, 16, 6, 247707, tzinfo=datetime.timezone.utc), bid=1687.7, bidSize=10.0, ask=1687.75, askSize=16.0, last=1687.7, lastSize=0.35757418, prevBidSize=7.0, volume=1913.34897409, high=1710.9, low=1661.2, close=1665.3, halted=0.0)]

In [24]:
util.df?

Signature: util.df(objs, labels: List[str] = None)
Docstring:
Create pandas DataFrame from the sequence of same-type objects.

Args:
  labels: If supplied, retain only the given labels and drop the rest.
File:      c:\users\kushwaha\anaconda3\lib\site-packages\ib_insync\util.py
Type:      function
